In [7]:
import os
import torch
from torchvision import models,transforms
from PIL import Image, ImageDraw
import torch.nn as nn
from annoy import AnnoyIndex

In [ ]:
image_folder = "PetImages/Dog"
image_files = os.listdir(image_folder)
image_files = [f for f in image_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

In [ ]:
weights = models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weights)
                                                                                            
model.fc = nn.Identity()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

annoy_index = AnnoyIndex(512,'angular')
annoy_index.load('dog_index.ann')


True

In [11]:
image_grid = Image.new('RGB',(1000,1000))

In [ ]:


grid_cols = 5
thumb_size = (200, 200)
grid_width = thumb_size[0] * grid_cols
grid_height = thumb_size[1] * ((24 + 1 + grid_cols - 1) // grid_cols)

for i, filename in enumerate(image_files):
    img_path = os.path.join(image_folder, filename)
    image = Image.open(img_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        if input_tensor.size(1) == 3:
            output_tensor = model(input_tensor)

            # Get 24 nearest neighbors
            nns = annoy_index.get_nns_by_vector(output_tensor[0], 24)

            # Create a blank canvas
            image_grid = Image.new('RGB', (grid_width, grid_height), 'white')

            # Draw red rectangle around the query image
            image_draw = ImageDraw.Draw(image)
            image_draw.rectangle([(0, 0), (thumb_size[0]-1, thumb_size[1]-1)], outline='red', width=8)

            # Paste the query image
            image = image.resize(thumb_size)
            image_grid.paste(image, (0, 0))

            # Paste the 24 neighbors
            for j, idx in enumerate(nns):
                neighbor_image = Image.open(os.path.join(image_folder, image_files[idx])).convert('RGB')
                neighbor_image = neighbor_image.resize(thumb_size)
                x = thumb_size[0] * ((j + 1) % grid_cols)
                y = thumb_size[1] * ((j + 1) // grid_cols)
                image_grid.paste(neighbor_image, (x, y))

            # Ensure the output folder exists
os.makedirs('ImageDump', exist_ok=True)

# Save the image grid
image_grid.save(f'ImageDump/image_{i}.png')

